#  Data Loading, Storage, and File Formats

o  Reading data and making it accessible (often called _data loading_) is a necessary first step for using most of the tools in this book. 

o  The term _parsing_ is also sometimes used to describe loading text data and interpreting it as tables and different data types. 

o  Input and output typically fall into a few main categories: 

   - reading text files and other more efficient on-disk formats
   - loading data from databases
   - interacting with network sources like web APIs.


### Reading and Writing Data in Text Format

o pandas features a number of functions for reading tabular data as a DataFrame object.

o Table 6.1: Text and binary data loading functions in pandas

o The functions all come with optional arguments for these functions may fall into a few categories:

   - Indexing - treat one or more columns as the returned DataFrame, and whether to get column names from the file, arguments you provide, or not at all.
   - Type inference and data conversion - includes the user-defined value conversions and custom list of missing value markers.
   - Date and time parsing - includes a combining capability, including combining date and time information spread over multiple columns into a single column in the result.
   - Iterating - Support for iterating over chunks of very large files.
   - Unclean data issues - includes skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.
    
o Some of these functions perform _type inference_, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, Boolean, or string. Other data formats, like HDF5, ORC, and Parquet, have the data type information embedded in the format.

### Comma-separated values (CSV) text file

o The pandas read_csv() function is the most common text file format for reading in tabular data

o The file parsing functions have many additional arguments to help handle the wide variety of exception file formats that occur 

o Table 6.2: Some pandas.read_csv function arguments


In [2]:
import pandas as pd

# read in cvs file in current folder and load into a pandas Dataframe
df = pd.read_csv("ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


o The first row is the header row which could cause a problem if the file does not have a header row.

In [2]:
pd.read_csv("ex2.csv")

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


o The _header_ argument can be used if a file does not always a header row.

In [3]:
pd.read_csv("ex2.csv", header=None)   # indicate that the first row in not the header (defaults to integer index)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_csv("ex2.csv", names=["a", "b", "c", "d", "mike"]) # provide a custom header

,a,b,c,d,mike
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


o Can also specify that one of the columns should be used as the _index_col_ argument:

In [6]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("ex2.csv", names=names, index_col="message")


,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


## Missing Values

o Handling missing values is an important and frequently nuanced part of the file reading process. 

o Missing data is usually either not present (empty string) or marked by some sentinel (placeholder) value such as NaN, NA and NULL.

o The *na_values* option accepts a sequence of strings to add to the default list of strings recognized as missing


In [9]:
result = pd.read_csv("ex5.csv")
result
#pd.isna(result)      # returns a Boolean DataFrame indicating valid/invalid values

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [10]:
result = pd.read_csv("ex5.csv", na_values=["NULL","foo"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## Reading Text Files in Pieces

o When processing very large files or figuring out the right set of arguments to correctly process a large file, you may want to read only a small piece of a file or iterate through smaller chunks of the file.

o To read a file in pieces, specify a chunksize as a number of rows:


In [ ]:
result = pd.read_csv("ex6.csv")
result

In [ ]:
chunker = pd.read_csv("ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot

## Writing Data to Text Format

o Data can also be exported to a delimited format. 

o The DataFrame’s _to_csv_ method can write the data out to a comma-separated file


In [11]:
data = pd.read_csv("ex5.csv", na_values=["NULL","foo"])
data.to_csv("out.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


o The default is that both the row and column labels are written. Both of these can be disabled

In [12]:
data.to_csv("out2.csv", index=False, header=False)

o Can also write a subset of the columns, and in any order:

In [13]:
data.to_csv("out3.csv", index=False, columns=["a", "b", "c"])

## Working with Other Delimited Formats

o For any file with a single-character delimiter, you can use Python’s built-in csv module. To use it, pass any open file or file-like object to csv.reader:

o From there, **data wrangling** is necessary to put the data in the form that is needed.  


In [ ]:
import csv

f = open("ex7.csv")
reader = csv.reader(f)
#Iterating through the reader like a file yields lists of values with any quote characters removed:
for line in reader:
	print(line)

f.close()

## JSON Data

o JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. 

o JSON is very nearly valid Python code with the exception of its null value null and some other nuances (such as disallowing trailing commas at the end of lists). 

o The basic types are objects (dictionaries), arrays (lists), strings, numbers, Booleans, and nulls. All of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. 

o It is a much more free-form data format than a tabular text form like CSV. Here is an example:


In [ ]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

## XML and HTML: Web Scraping (?)

o Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples include lxml, Beautiful Soup, and html5lib. 

o pandas has a built-in function, *pandas.read_html*, which uses all of these libraries to automatically parse tables out of HTML files as DataFrame objects. 

o The pandas.read_html function has a number of options, but by default it searches for and attempts to parse all tabular data contained within <table> tags. The result is a list of DataFrame objects:

In [14]:
tables = pd.read_html("fdic_failed_bank_list.html")
print(len(tables))
failures = tables[0]
failures.head()

1


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


## Binary Data Formats

o One simple way to store (or serialize) data in binary format is using Python’s built-in *pickle* module. 

o pandas objects all have a *to_pickle* method that writes the data to disk in pickle format:

o Pickle files are in general readable only in Python. You can read any "pickled" object stored in a file by using the built-in pickle directly, or even more conveniently using *pandas.read_pickle*

o Danger - pickle is recommended only as a short-term storage format. The problem is that it is hard to guarantee that the format will be stable over time; an object pickled today may not unpickle with a later version of a library. 

o pandas has built-in support for several other open source binary data formats, such as HDF5, ORC, and Apache Parquet. 


In [15]:
frame = pd.read_csv("ex1.csv")
frame.to_pickle("frame_pickle")
pd.read_pickle("frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Reading Microsoft Excel Files

o pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the pandas *ExcelFile* class or *pandas.read_excel* function. 

o If you are not reading multiple sheets use pd.read_excel.

o Still can read individual sheets from a multiple sheet Excel file using *sheet_name* optional argument


In [16]:
frame = pd.read_excel("ex1.xlsx")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [17]:
frame = pd.read_excel("ex1.xlsx", sheet_name="Sheet1")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


o When reading multiple sheets in a file,  it is faster to create the pandas.ExcelFile, 

In [18]:
xlsx = pd.ExcelFile("ex1.xlsx")
xlsx.sheet_names

['Sheet1']

In [19]:
# Data stored in a sheet can then be read into DataFrame with parse:
xlsx.parse(sheet_name="Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [20]:
# This Excel table has an index column, so we can indicate that with the index_col argument
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Writing Microsoft Excel Files

o For multiple sheet Excel files, to write pandas data to Excel format, first create an ExcelWriter, then write data to it using the pandas object's to_excel method:


In [27]:
writer = pd.ExcelWriter("ex2.xlsx")
frame.to_excel(writer, "Sheet1")
#writer.save()                     # this has been depreciated
frame.to_excel("ex2.xlsx")

o For a single sheet, can do in one line

In [30]:
frame.to_excel("output.xlsx")

## Using HDF5 Format

o HDF5 is a respected file format intended for storing large quantities of scientific array data. It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python. 

o The “HDF” in HDF5 stands for hierarchical data format. Each HDF5 file can store multiple datasets and supporting metadata. Compared with simpler formats, HDF5 supports on-the-fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently. 

o HDF5 can be a good choice for working with datasets that don't fit into memory, as you can efficiently read and write small sections of much larger arrays.

## Interacting with Web APIs

o Many websites have public APIs providing data feeds via JSON or some other format. 

o There are a number of ways to access these APIs from Python

## Interacting with Databases

o In a business setting, a lot of data may not be stored in text or Excel files. 

o SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. 

o The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application.
pandas has some functions to simplify loading the results of a SQL query into a DataFrame. 
